# 1. Preprocess-Stage1-Group-All-TrainData

## Run name

In [1]:
import time
project_name = 'Google-LandMark-Rec2019'
step_name = '1-Preprocess-Stage1-Group-All-TrainData'
time_str = time.strftime("%Y%m%d-%H%M%S", time.localtime())
run_name = project_name + '_' + step_name + '_' + time_str
print('run_name: ' + run_name)
t0 = time.time()

run_name: Google-LandMark-Rec2019_1-Preprocess-Stage1-Group-All-TrainData_20190424-152705


## Important params

In [2]:
import multiprocessing

cpu_amount = multiprocessing.cpu_count()
# train_tar_count = 500

# print('train_tar_count:', train_tar_count)
print('cpu_amount: ', cpu_amount)

cpu_amount:  1


## Import PKGs

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline
from IPython.display import display

import os, sys, gc, math, shutil, zipfile, pickle, h5py, re, tarfile
import urllib, xlsxwriter
from tqdm import tqdm
import hashlib

## Project folders

In [4]:
cwd = os.getcwd()
feature_folder = os.path.join(cwd, 'feature')
input_folder = os.path.join(cwd, 'input')
output_folder = os.path.join(cwd, 'output')
model_folder = os.path.join(cwd, 'model')

md5sum_folder = os.path.join(input_folder, 'md5sum')
if not os.path.exists(md5sum_folder):
    os.mkdir(md5sum_folder)
    print('create folder:', md5sum_folder)
    
train_tar_folder = os.path.join(input_folder, 'train_tar')
if not os.path.exists(train_tar_folder):
    os.mkdir(train_tar_folder)
    print('create folder:', train_tar_folder)
    
train_untar_folder = os.path.join(input_folder, 'train_untar')
if not os.path.exists(train_untar_folder):
    os.mkdir(train_untar_folder)
    print('create folder:', train_untar_folder)
    
all_image_folder = os.path.join(input_folder, 'all_image')
if not os.path.exists(all_image_folder):
    os.mkdir(all_image_folder)
    print('create folder:', all_image_folder)
    
all_train_data_folder = os.path.join(input_folder, 'all_train_data')
if not os.path.exists(all_train_data_folder):
    os.mkdir(all_train_data_folder)
    print('create folder:', all_train_data_folder)

In [5]:
!pwd
!ls ./input -hl

/data/landmark-recognition-2019
total 1.1G
drwxrwxr-x   2 ubuntu ubuntu 161M Apr 24 14:23 all_image
drwxrwxr-x   2 ubuntu ubuntu 4.0K Apr 24 14:42 all_train_data
-rwxrwxrwx   1 ubuntu ubuntu 127M Apr 20 12:00 data_train.csv
-rwxrwxrwx   1 ubuntu ubuntu 108M Apr 21 09:40 google-landmarks-dataset.zip
-rwxrwxrwx   1 ubuntu ubuntu 1.9M Apr 20 12:00 google_landmark_boxes_split1.csv
-rwxrwxrwx   1 ubuntu ubuntu 1.2M Apr 20 12:00 google_landmark_boxes_split2.csv
-rwxrwxrwx   1 ubuntu ubuntu 108M Apr 20 12:00 index.csv
drwxrwxrwx   2 ubuntu ubuntu  20K Apr 21 10:10 md5sum
-rwxrwxrwx   1 ubuntu ubuntu 3.0M Apr  8 02:17 recognition_sample_submission.csv
-rwxrwxrwx   1 ubuntu ubuntu 1.3M Apr 21 09:25 recognition_sample_submission.csv.zip
-rwxrwxrwx   1 ubuntu ubuntu  15M Apr 20 12:00 test.csv
-rw-rw-r--   1 ubuntu ubuntu 502M Apr  5 21:57 train.csv
drwxrwxrwx   2 ubuntu ubuntu  20K Apr 23 16:04 train_tar
drwxrwxrwx 502 ubuntu ubuntu  20K Apr 22 22:57 train_untar


In [6]:
# !ls ./input/all_image/ -l|wc -l

In [7]:
# !wget https://s3.amazonaws.com/google-landmark/metadata/train.csv

In [8]:
train_csv = os.path.join(input_folder, 'train.csv')
test_csv = os.path.join(input_folder, 'test.csv')
boxes_split1_csv = os.path.join(input_folder, 'google_landmark_boxes_split1.csv')
boxes_split2_csv = os.path.join(input_folder, 'google_landmark_boxes_split2.csv')
recognition_sample_submission_csv = os.path.join(input_folder, 'recognition_sample_submission.csv')

print(train_csv)
print(test_csv)
print(boxes_split1_csv)
print(boxes_split2_csv)
print(recognition_sample_submission_csv)


/data/landmark-recognition-2019/input/train.csv
/data/landmark-recognition-2019/input/test.csv
/data/landmark-recognition-2019/input/google_landmark_boxes_split1.csv
/data/landmark-recognition-2019/input/google_landmark_boxes_split2.csv
/data/landmark-recognition-2019/input/recognition_sample_submission.csv


## Preview csv

In [9]:
%%time
train_df = pd.read_csv(train_csv, engine='python', nrows=200*10000)
display(train_df.shape, train_df.head(2))

(1000000, 3)

,id,url,landmark_id
0,6e158a47eb2ca3f6,https://upload.wikimedia.org/wikipedia/commons...,142820
1,202cd79556f30760,http://upload.wikimedia.org/wikipedia/commons/...,104169


CPU times: user 4.24 s, sys: 365 ms, total: 4.61 s
Wall time: 9.61 s


In [10]:
index_train_df = train_df.set_index(['id'], drop=True)
display(index_train_df.shape, index_train_df.head())

(1000000, 2)

,url,landmark_id
id,,
6e158a47eb2ca3f6,https://upload.wikimedia.org/wikipedia/commons...,142820
202cd79556f30760,http://upload.wikimedia.org/wikipedia/commons/...,104169
3ad87684c99c06e1,http://upload.wikimedia.org/wikipedia/commons/...,37914
e7f70e9c61e66af3,https://upload.wikimedia.org/wikipedia/commons...,102140
4072182eddd0100e,https://upload.wikimedia.org/wikipedia/commons...,2474


In [11]:
'6799cb2970362f33' in index_train_df.index

False

## Create soft link

In [12]:
min_id = min(train_df['landmark_id'])
max_id = max(train_df['landmark_id'])
print(min_id, max_id)

0 203093


In [13]:
%%time
ivalid_image_count = 0
for _, row in train_df[:10].iterrows():
    src_image_path = os.path.join(all_image_folder, '%s.jpg' % row['id'])
    print(src_image_path)
    if not os.path.exists(src_image_path):
        ivalid_image_count += 1
        continue
    print(src_image_path)
    target_image_folder = os.path.join(all_train_data, '%06d' % row['landmark_id'])
    print(target_image_folder)
    if not os.path.exists(target_image_folder):
#         os.mkdir(target_image_folder)
        print('Folder created:')
#     target_image_path = 

print('ivalid_image_count:', ivalid_image_count)

/data/landmark-recognition-2019/input/all_image/6e158a47eb2ca3f6.jpg
/data/landmark-recognition-2019/input/all_image/6e158a47eb2ca3f6.jpg


NameError: name 'all_train_data' is not defined

In [14]:
# !ls ./input/all_image -U|head -5

In [15]:
# 6799cb2970362f33.jpg
# 6e60619317ab4083.jpg
# a2e31f0268dac709.jpg
# 1fa63b7e815a5183.jpg
# 6b1701ae0accdf44.jpg

In [16]:
print(src_image_path)

/data/landmark-recognition-2019/input/all_image/6e158a47eb2ca3f6.jpg


In [17]:
# %%time
# for i in range(train_tar_count):
#     image_folder = os.path.join(train_untar_folder, 'images_%03d' % i)
#     if os.path.exists(image_folder):
#         src_image_arr = get_all_images_from_folder(image_folder)
#         for src_image_path in src_image_arr:
#             image_name = src_image_path.split('/')[-1]
#             target_image_path = os.path.join(all_image_folder, image_name)
# #             print(src_image_path, target_image_path)
# #         print(image_folder, len(src_image_arr))
#             if not os.path.exists(target_image_path):
#                 os.symlink(src_image_path, target_image_path)
#             continue

In [18]:
len('6799cb2970362f33.jpg')

20

In [19]:
len('3e92de9660be546b.jpg')

20

In [20]:
print('Time elapsed: %.1fs' % (time.time() - t0))
print(run_name)

Time elapsed: 12.4s
Google-LandMark-Rec2019_1-Preprocess-Stage1-Group-All-TrainData_20190424-152705
